In [12]:
import pandas as pd
import pandas_ta as ta
import os
# Create your own Custom Strategy
CustomStrategy = ta.Strategy(
    name="Momo and Volatility",
    description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
    ta=[
        {"kind": "ema", "length": 8},
        {"kind": "ema", "length": 21},
        {"kind": "ema", "length": 34},
        {"kind": "ema", "length": 55},
        {"kind": "sma", "length": 89},
        {"kind": "bbands", "length": 20},
        {"kind": "kc", "length": 20},
		{"kind": "atr", "length": 20},
        {"kind": "rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
        {"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    ]
)

df = pd.read_csv('datasets/CNP.csv.gz')
df.ta.strategy(CustomStrategy)


In [2]:

def is_ema_stacked(df):
    avg = [
        df.ta.ema(8).iloc[-1],
        df.ta.ema(21).iloc[-1],
        df.ta.ema(34).iloc[-1]
    ]
    return avg[0] > avg[1] and avg[1] > avg[2]

def inspect_squeeze(symbol):
    df = pd.read_csv('datasets/CNP.csv.gz')
    status_spro = 0
    status_ema = 0
    status_adx = 0
    status_buy_zone = 0

    spro = df.ta.squeeze_pro()
    if spro.iloc[-4:].SQZPRO_ON_NORMAL.sum() == 4:
        status_spro = 1

    if is_ema_stacked(df):
        status_ema = 1

    adx = df.ta.adx().ADX_14.iloc[-1]
    if adx < 20.0:
        status_adx = 1

    if df.ta.ema(21).iloc[-1] * 1.05 >  df.iloc[-1].close:
        status_buy_zone = 1
    bar = df.iloc[-1]

    rsi = df.ta.rsi().iloc[-1]
    atr = df.ta.atr().iloc[-1]
    sell1= bar.close + (atr*2)
    est_ret = (atr*2)/bar.close*100
    str = f'atr={atr:05.2f} rsi={rsi:5.2f} close={bar.close:6.2f} sell1={sell1:6.2f} est_ret={est_ret:05.2f}'

    status_verdict = status_spro + status_ema + status_adx + status_buy_zone
    print(f"{symbol:4s} spro={status_spro} ema={status_ema} adx={adx:2.0f} buy_zone={status_buy_zone} verdict={status_verdict} {str}")


In [10]:
def load_symbol(symbol):
    file = f"datasets/{symbol}.csv.gz"
    if os.path.isfile(file):
        df = pd.read_csv(file)
    else:
        start_dt = datetime.datetime.now() - datetime.timedelta(days = 50)
        start_str = start_dt.strftime("%Y-%m-%d")
        df = yf.download(symbol, progress=False, period='3mo') #start=start_str)
        df.columns = df.columns.str.lower()
        df.to_csv(file,index=True)
    df.name = symbol
    return df

# df = pd.read_csv('datasets/CNP.csv.gz')
# df = load_symbol('CNP')
xdf = df.ta.cdl_pattern(name='all')
#inspect_squeeze('TQQQ')
print(xdf.iloc[-1])


CDL_2CROWS              0.0
CDL_3BLACKCROWS         0.0
CDL_3INSIDE             0.0
CDL_3LINESTRIKE         0.0
CDL_3OUTSIDE            0.0
                       ... 
CDL_THRUSTING           0.0
CDL_TRISTAR             0.0
CDL_UNIQUE3RIVER        0.0
CDL_UPSIDEGAP2CROWS     0.0
CDL_XSIDEGAP3METHODS    0.0
Name: 63, Length: 62, dtype: float64


In [15]:
df = load_symbol('AAPL')
xdf = df.ta.cdl_pattern(name='all')
xdf

,CDL_2CROWS,CDL_3BLACKCROWS,CDL_3INSIDE,CDL_3LINESTRIKE,CDL_3OUTSIDE,CDL_3STARSINSOUTH,CDL_3WHITESOLDIERS,CDL_ABANDONEDBABY,CDL_ADVANCEBLOCK,CDL_BELTHOLD,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
ema = pd.concat({
    'ema8': df.ta.ema(8),
    'ema21': df.ta.ema(21),
    'ema34': df.ta.ema(34)
    }, axis=1)
ema['stack_bull'] = (ema.ema8 > ema.ema21) & (ema.ema21 > ema.ema34)
ema['stack_bear'] = (ema.ema8 < ema.ema21) & (ema.ema21 < ema.ema34)
ema
#help(ta.ema)

,ema8,ema21,ema34,stack_bull,stack_bear
0,NaN,NaN,NaN,False,False
1,NaN,NaN,NaN,False,False
2,NaN,NaN,NaN,False,False
3,NaN,NaN,NaN,False,False
4,NaN,NaN,NaN,False,False
...,...,...,...,...,...
59,145.092783,139.751906,136.898972,True,False
60,145.381054,140.355369,137.441317,True,False
61,144.729708,140.545790,137.727527,True,False
62,145.045327,141.055263,138.208811,True,False
